In [1]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value, current_dow30_symbols
import numpy as np
import pandas as pd

In [8]:
symbol="AMD"
timeInterval = "1d"
longTimeInterval = "1d"
timeRange = "5y"

In [9]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'AMD',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 322151400,
 'regularMarketTime': 1614373203,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 84.51,
 'chartPreviousClose': 2.07,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '5y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [10]:
longMetadataAndPrice = metadata_price_action(symbol, longTimeInterval, timeRange)
longMetadata = metadataAndPrice["metadata"]
longMetadata

{'currency': 'USD',
 'symbol': 'AMD',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 322151400,
 'regularMarketTime': 1614373203,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 84.51,
 'chartPreviousClose': 2.07,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '5y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [11]:
price = metadataAndPrice["price"]
price.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
price = price.dropna()
price

,timestamp,Close,Open,High,Volume,Low
0,1456756200,2.060000,2.140000,2.150000,11400300,2.050000
1,1456842600,2.160000,2.180000,2.200000,9011200,2.120000
2,1456929000,2.200000,2.320000,2.330000,10845700,2.180000
3,1457015400,2.320000,2.460000,2.550000,22092400,2.300000
4,1457101800,2.420000,2.370000,2.500000,15449000,2.340000
...,...,...,...,...,...,...
1254,1614004200,88.150002,85.370003,88.300003,36804800,85.209999
1255,1614090600,83.400002,84.739998,85.110001,49690200,79.360001
1256,1614177000,84.330002,86.940002,87.089996,40725200,82.809998
1257,1614263400,86.169998,82.419998,87.089996,54427700,81.919998


In [12]:
longPrice = longMetadataAndPrice["price"]
longPrice.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
longPrice

,timestamp,Close,Open,High,Volume,Low
0,1456756200,2.060000,2.140000,2.150000,11400300,2.050000
1,1456842600,2.160000,2.180000,2.200000,9011200,2.120000
2,1456929000,2.200000,2.320000,2.330000,10845700,2.180000
3,1457015400,2.320000,2.460000,2.550000,22092400,2.300000
4,1457101800,2.420000,2.370000,2.500000,15449000,2.340000
...,...,...,...,...,...,...
1254,1614004200,88.150002,85.370003,88.300003,36804800,85.209999
1255,1614090600,83.400002,84.739998,85.110001,49690200,79.360001
1256,1614177000,84.330002,86.940002,87.089996,40725200,82.809998
1257,1614263400,86.169998,82.419998,87.089996,54427700,81.919998


In [13]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from statistics import mean

def mySMA(myArr, period):
    return pd.Series(myArr).rolling(period).mean()
            
def myEMA(myArr, period):
    return pd.Series(myArr).ewm(adjust=False, span=period).mean()
    
def myMacd(myArr):
    twelve = myEMA(myArr, 12) 
    twentySix = myEMA(myArr, 26)
    macd = twelve-twentySix
#     print(macd, twelve, twentySix)
    return macd

def myMacdSignal(myArr):
    return myEMA(myMacd(myArr), 9)

def myMacdHist(myArr):
    return myMacd(myArr) - myMacdSignal(myArr)
    
class SmaCross(Strategy):
    def init(self):
        tempprice = self.data.Close
        print("buy and hold range", tempprice[0], tempprice[-1])
        self.ma1 = self.I(myEMA, tempprice, 9)
        self.ma2 = self.I(myEMA, tempprice, 12)
        self.macd = self.I(myMacd, tempprice)
        self.macdSignal = self.I(myMacdSignal, tempprice)
        self.macdHist = self.I(myMacdHist, tempprice)

    def next(self):
        if crossover(self.macd, self.macdSignal):
            self.buy()
        elif crossover(self.macdSignal, self.macd):
            self.sell()


bt = Backtest(price, SmaCross,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

<ipython-input-13-f469da29355e>:42: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(price, SmaCross,


buy and hold range 2.059999942779541 83.56999969482422


In [38]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '82.42',
 'Open': '83.57',
 'Bid': '84.30 x 900',
 'Ask': '84.42 x 800',
 "Day's Range": '82.91 - 85.59',
 '52 Week Range': '36.75 - 99.23',
 'Volume': '48,962,907',
 'Avg. Volume': '43,362,416',
 'Market Cap': '102.369B',
 'Beta (5Y Monthly)': '2.20',
 'PE Ratio (TTM)': '40.94',
 'EPS (TTM)': '2.06',
 'Earnings Date': 'Apr 26, 2021 - Apr 30, 2021',
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Ex-Dividend Date': 'Apr 27, 1995'}

In [39]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['102.65B',
  '111.16B',
  '98.55B',
  '61.76B',
  '53.26B',
  '53.66B'],
 'Enterprise Value 3': ['100.89B',
  '109.97B',
  '97.67B',
  '61.08B',
  '52.48B',
  '53.56B'],
 'Trailing P/E ': ['41.14', '123.93', '157.67', '122.35', '151.60', '241.37'],
 'Forward P/E 1': ['44.44', '51.02', '49.26', '52.63', '39.84', '43.48'],
 'PEG Ratio (5 yr expected) 1': ['1.58',
  '1.13',
  '1.42',
  '1.55',
  '1.66',
  '1.55'],
 'Price/Sales (ttm)': ['10.48', '12.68', '12.67', '8.37', '7.57', '8.38'],
 'Price/Book (mrq)': ['17.59', '28.72', '29.82', '20.34', '18.84', '24.66'],
 'Enterprise Value/Revenue 3': ['10.33',
  '33.90',
  '34.87',
  '31.61',
  '29.39',
  '25.18'],
 'Enterprise Value/EBITDA 6': ['60.20',
  '167.63',
  '193.41',
  '238.59',
  '202.63',
  '180.94'],
 'Beta (5Y Monthly) ': '2.20',
 '52-Week Change 3': '78.07%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '99.23',
 '52 Week Low 3': '36.75',
 '50-Day Moving Average 3': '89.89',
 '200-Day Movi